In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import time
import rebound
import matplotlib.pyplot as plt
import numpy as np
import eccentricWH_bruteforce
import vanillaWH_tools

%matplotlib inline

# configuring plots
#plt.rc('text', usetex=True)            # use LaTeX for text
plt.rc('font', family='serif')          # use serif font
plt.rcParams.update({'font.size': 14})  # increase font size

SyntaxError: invalid syntax (eccentricWH_bruteforce.py, line 57)

In [5]:
c = np.array([1,2])
a, b = c
a+=1
b+=1
c = np.array([a,b])

print(c)


[2 3]


In [ ]:
# initial conditions from Mikkola 1997
# particle array as [x, y, z, vx, vy, vz, m]
p0 = np.array([0., 0., 0., 0., 0., 0., 1.])
p1 = np.array([-2.0, 0., 0., 0., -0.7, 0., 2.])
p2 = np.array([0., 60., 0.01, -0.095, 0.0, 0.01, 0.9])

sim_jacobi = np.stack([p0, p1, p2])
masses = sim_jacobi[:,-1]

sim = vanillaWH_tools.jacobi_to_inertial(sim_jacobi, masses)

P_1195 = 2*np.pi*np.sqrt(1.195219123505976**3 / 3)
print(P_1195)

P_1195 = 4.73

print(P_1195)
print('Period =', P_1195)

timestep = 1./30. * P_1195
N_steps = 12500

energy_error = np.zeros(N_steps)
E0 = vanillaWH_tools.energy_fn(sim)

orbit_soln = np.zeros((len(sim_jacobi), 3, N_steps)) # [object, position, N_steps]
orbit_soln[:,0,0], orbit_soln[:,1,0], orbit_soln[:,2,0] = sim_jacobi[:,0], sim_jacobi[:,1], sim_jacobi[:,2]

start = time.time()

for i in range(N_steps):
    
    # drift
    sim_jacobi = eccentricWH_bruteforce.drift(sim_jacobi, sim, object_num = 1, h=timestep/2)
    sim_jacobi = eccentricWH_bruteforce.drift(sim_jacobi, sim, object_num = 2, h=timestep/2)
    sim = vanillaWH_tools.jacobi_to_inertial(sim_jacobi, masses)
    
    # kick
    sim_jacobi = eccentricWH_bruteforce.kick(sim_jacobi, sim, h=timestep)
    
    # drift
    sim_jacobi = eccentricWH_bruteforce.drift(sim_jacobi, sim, object_num = 1, h=timestep/2)
    sim_jacobi = eccentricWH_bruteforce.drift(sim_jacobi, sim, object_num = 2, h=timestep/2)
    sim = vanillaWH_tools.jacobi_to_inertial(sim_jacobi, masses)

    E = vanillaWH_tools.energy_fn(sim)
    energy_error[i] = (E-E0)/E0

    orbit_soln[:, 0, i] = sim_jacobi[:,0]
    orbit_soln[:, 1, i] = sim_jacobi[:,1]
    orbit_soln[:, 2, i] = sim_jacobi[:,2]

end = time.time()

print('time to integrate =', np.round(end - start, 1), 's')